# Imports e definições gerais

In [3]:
import pandas as pd
from dash import Dash, html, dcc
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [4]:
ARQUIVO_CSV = 'dialise_fav_final3.csv'
SEPARADOR   = ';'

df = pd.read_csv(ARQUIVO_CSV, sep=SEPARADOR)
df.sample(5)

/tmp/ipykernel_26421/3187438065.py:4: DtypeWarning: Columns (20,37,104) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ARQUIVO_CSV, sep=SEPARADOR)


: 

In [ ]:
paleta = {
    'azul_principal': '#0077B6',
    'azul_claro'    : '#90E0EF',
    'ciano'         : '#48CAE4',
    'azul_escuro'   : '#023E8A',
    'cinza_texto'   : '#6C757D',
    'fundo'         : '#FFFFFF',
    'cinza_claro'   : '#F8F9FA'
}

# Tratamento de Dados

In [ ]:
df_espera = pd.DataFrame()

# Pergunta 1 e 2
df_espera['data_primeira_dialise']      = df['ATD_DTPDR']
df_espera['cronico']                    = df['Cronico']
df_espera['data_atendimento_dialitico'] = df['AP_CMP']
df_espera['obito']                      = df['AP_OBITO']
df_espera['data_atendimento_fav']       = df['fav_AP_CMP']
df_espera.sample(10)

# Pergunta 3
df_espera['genero'] = df['AP_SEXO']
df_espera['idade'] = df['AP_NUIDADE']

# Pergunta 4
df_espera['hospital'] = df['fav_AP_CODUNI']

In [ ]:
# Para os casos de morte, podemos desconsiderar o tempo de espera (?)
df_espera_mortos_validos = df_espera[df_espera.obito == 1][df_espera.data_atendimento_fav.notna()]

# Remover todos os mortos
df_espera = df_espera[df_espera.obito != 1]

# Inserir apenas falecidos válidos
df_espera = pd.concat([df_espera, df_espera_mortos_validos])
df_espera.sample(10)

/tmp/ipykernel_18655/2896329900.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav,genero,idade,hospital
602417,2021-01-01,repetido,2022-12-01,0,NaN,F,23,NaN
523158,2021-06-22,repetido,2021-12-01,0,2020-09-01,M,59,2242397.0
424472,2020-07-09,agudo,2020-08-01,0,2020-08-01,F,47,2257815.0
283221,2015-11-05,repetido,2018-08-01,0,2015-11-01,F,61,2226952.0
584104,2022-09-07,agudo,2022-09-01,0,NaN,F,34,NaN
137489,2015-12-17,repetido,2016-07-01,0,2015-03-01,F,52,2253046.0
605152,2019-01-26,repetido,2023-01-01,0,2021-05-01,F,46,2230577.0
137258,1997-08-01,repetido,2016-07-01,0,NaN,M,58,NaN
338540,2018-01-17,repetido,2019-05-01,0,2018-05-01,F,50,2261057.0
235624,2016-01-14,repetido,2017-12-01,0,2016-01-01,M,57,2237253.0


In [ ]:
# Priorizar remover repetidos NaN
df_espera = df_espera[df_espera.cronico != 'repetido']
df_espera.sample(5)

,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav,genero,idade,hospital
1761,2013-07-03,cronico,2014-08-01,0,NaN,F,66,NaN
527397,2021-10-14,agudo,2021-12-01,0,2021-11-01,F,63,2255456.0
195539,2017-05-29,agudo,2017-05-01,0,NaN,M,70,NaN
70942,2015-07-31,agudo,2015-08-01,0,NaN,F,65,NaN
114245,2008-05-26,cronico,2016-03-01,0,2014-09-01,M,55,2237601.0


In [ ]:
# Remover observações que não são agudos e que não tiveram atendimento FAV
# Para garantir, vamos remover todos que não tiveram data_primeira_dialise também
df_espera = df_espera[df_espera.cronico != 'agudo']
df_espera = df_espera[df_espera.data_atendimento_fav.notna()]
df_espera = df_espera[df_espera.data_primeira_dialise.notna()]
df_espera.sample(10)

,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav,genero,idade,hospital
23537,2014-09-12,cronico,2014-12-01,0,2014-10-01,M,46,2253046.0
76756,2015-06-25,cronico,2015-09-01,0,2015-11-01,F,53,2226952.0
628504,2023-01-17,cronico,2023-04-01,0,2023-06-01,F,44,2246988.0
355619,2019-05-17,cronico,2019-08-01,0,2019-06-01,M,60,2255456.0
356424,2019-03-02,cronico,2019-08-01,0,2019-10-01,M,76,2700662.0
448802,2020-08-14,cronico,2020-11-01,0,2020-09-01,M,62,2262460.0
86898,2015-07-27,cronico,2015-10-01,0,2015-08-01,F,45,2247968.0
232362,2017-09-22,cronico,2017-12-01,0,2018-06-01,F,56,2262584.0
683702,2023-10-21,cronico,2024-01-01,0,2024-02-01,F,34,2262460.0
34384,2014-10-31,cronico,2015-01-01,0,2015-03-01,M,41,2262509.0


In [ ]:
# Calcular tempo de espera
df_espera['data_atendimento_fav'] = pd.to_datetime(df_espera['data_atendimento_fav'])
df_espera['data_primeira_dialise'] = pd.to_datetime(df_espera['data_primeira_dialise'])

# Calcular a diferença
df_espera['tempo_entre_fav_e_primeira_dialise'] = (
    df_espera['data_atendimento_fav'] - df_espera['data_primeira_dialise']
).dt.days  # em dias

df_espera['tempo_anos'] = df_espera['tempo_entre_fav_e_primeira_dialise'] / 365.25

# Remover esperas negativas
df_espera = df_espera[df_espera['tempo_entre_fav_e_primeira_dialise'] > 0]
df_espera.sample(10)

,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav,genero,idade,hospital,tempo_entre_fav_e_primeira_dialise,tempo_anos
33243,2014-10-10,cronico,2015-01-01,0,2021-05-01,M,57,2256029.0,2395,6.557153
1717,2012-04-18,cronico,2014-08-01,0,2016-07-01,M,65,2253054.0,1535,4.202601
675382,2023-08-19,cronico,2023-11-01,0,2023-10-01,F,66,2255456.0,43,0.117728
4565,2010-08-27,cronico,2014-08-01,0,2014-08-01,M,56,2262460.0,1435,3.928816
379266,2019-09-19,cronico,2019-12-01,0,2019-10-01,F,65,2230577.0,12,0.032854
95943,2015-09-24,cronico,2015-12-01,0,2015-11-01,F,58,2262568.0,38,0.104038
515385,2021-07-06,cronico,2021-10-01,0,2021-10-01,F,49,2227290.0,87,0.238193
664721,2023-07-14,cronico,2023-10-01,0,2023-10-01,F,46,2262509.0,79,0.216290
484816,2021-02-23,cronico,2021-05-01,0,2021-04-01,M,77,2236370.0,37,0.101300
4178,2011-06-18,cronico,2014-08-01,0,2015-02-01,F,55,2261898.0,1324,3.624914


In [1]:
# Manter dados apenas a partir de janeiro de 2014 devido criação do dataset
df_espera['data_atendimento_dialitico'] = pd.to_datetime(df_espera['data_atendimento_dialitico'])
df_espera['data_atendimento_fav'] = pd.to_datetime(df_espera['data_atendimento_fav'])
df_espera['data_primeira_dialise'] = pd.to_datetime(df_espera['data_primeira_dialise'])

df_espera = df_espera[
        (df_espera['data_atendimento_dialitico'] >= '2014-01-01') &
        (df_espera['data_atendimento_fav'] >= '2014-01-01') &
        (df_espera['data_primeira_dialise'] >= '2014-01-01') ]

NameError: name 'pd' is not defined

In [19]:
df_perfil = df_espera[(df_espera.genero.notna()) & (df_espera.idade.notna())]

In [21]:
df_hospital = df_espera[df_espera.hospital.notna()]
df_hospital.sample()

,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav,genero,idade,hospital,tempo_entre_fav_e_primeira_dialise,tempo_anos
177215,2016-11-03,cronico,2017-02-01,0,2016-12-01,F,26,2256029.0,28,0.07666


# 1) Após o início da diálise crônica, qual é o tempo médio até a confecção da Fístula Arterial Venosa (FAV) nos hospitais de POA/RS?

In [12]:
# Gráfico de tempo de espera
max_tempo = df_espera['tempo_entre_fav_e_primeira_dialise'].max()
bins = list(range(0, int(max_tempo) + 10, 10))

# Plotar histograma
fig1_p1 = px.histogram(
    df_espera,
    x='tempo_entre_fav_e_primeira_dialise',
    nbins=len(bins),
    title='Distribuição do Tempo de Espera para Atendimento no FAV',
    labels={
        'tempo_entre_fav_e_primeira_dialise': 'Tempo de espera (dias)',
        'count': 'Quantidade de pessoas'
    },
    color_discrete_sequence=[paleta['azul_principal']]  # Cor das barras
)

# Layout personalizado com a paleta
fig1_p1.update_layout(
    xaxis_title='Tempo de espera (dias)',
    yaxis_title='Quantidade de pessoas',
    bargap=0.05,
    template='plotly_white',
    plot_bgcolor=paleta['fundo'],
    paper_bgcolor=paleta['fundo'],
    font=dict(color=paleta['azul_escuro']),
    title_font=dict(size=18, color=paleta['azul_escuro']),
    xaxis=dict(
        gridcolor=paleta['cinza_claro'],
        linecolor=paleta['cinza_texto'],
        tickfont=dict(color=paleta['azul_escuro'])
    ),
    yaxis=dict(
        gridcolor=paleta['cinza_claro'],
        linecolor=paleta['cinza_texto'],
        tickfont=dict(color=paleta['azul_escuro'])
    )
)

fig1_p1.show()

In [14]:
df_espera['ano_primeira_dialise'] = df_espera['data_primeira_dialise'].dt.year
df_agrupado = df_espera.groupby('ano_primeira_dialise')['tempo_entre_fav_e_primeira_dialise'].mean().reset_index()

fig2_p1 = px.bar(
    df_agrupado,
    x='ano_primeira_dialise',
    y='tempo_entre_fav_e_primeira_dialise',
    text='tempo_entre_fav_e_primeira_dialise',
    labels={
        'ano_primeira_dialise': 'Ano da Primeira Diálise',
        'tempo_entre_fav_e_primeira_dialise': 'Tempo médio de espera (dias)'
    },
    title='Tempo Médio de Espera por Ano da Primeira Diálise até a FAV',
    color='tempo_entre_fav_e_primeira_dialise',
    color_continuous_scale='Blues'
)

# Personalizar layout
fig2_p1.update_traces(texttemplate='%{text:.0f} dias', textposition='outside')

fig2_p1.update_layout(
    xaxis=dict(tickmode='linear'),
    yaxis_title='Tempo médio de espera (dias)',
    xaxis_title='Ano da primeira diálise',
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    plot_bgcolor='white',
    width=900,
    height=500
)

fig2_p1.show()


# 2) Houve mudanças na demora ao longo dos anos, especialmente em períodos críticos como a pandemia de COVID-19?

In [ ]:
# Garantir que a coluna data_primeira_dialise está em datetime
df_espera['data_primeira_dialise'] = pd.to_datetime(df_espera['data_primeira_dialise'])

# Criar coluna de ano-mês
df_espera['ano_mes'] = df_espera['data_primeira_dialise'].dt.to_period('M').astype(str)

# Agrupar por ano-mês
df_mes = df_espera.groupby('ano_mes')['tempo_entre_fav_e_primeira_dialise'].mean().reset_index()

# Converter ano-mês para datetime (para plotar no eixo)
df_mes['ano_mes_dt'] = pd.to_datetime(df_mes['ano_mes'])

# Gráfico de Linha (sem pontos)
fig1_p2 = go.Figure()

fig1_p2.add_trace(go.Scatter(
    x=df_mes['ano_mes_dt'],
    y=df_mes['tempo_entre_fav_e_primeira_dialise'],
    mode='lines',
    name='Tempo médio de espera',
    line=dict(color=paleta['azul_principal'], width=3)
))

# Tarja vermelha para COVID-19 (março de 2020 até dezembro de 2021)
fig1_p2.add_vrect(
    x0="2020-03-01", x1="2021-12-31",
    fillcolor="red", opacity=0.2,
    layer="below", line_width=0,
    annotation_text="COVID-19", annotation_position="top left",
    annotation_font_size=12, annotation_font_color="red"
)

# Layout final com paleta aplicada
fig1_p2.update_layout(
    title='Média do Tempo de Espera por Mês',
    xaxis_title='Ano',
    yaxis_title='Tempo médio de espera (dias)',
    template='plotly_white',
    plot_bgcolor=paleta['fundo'],
    paper_bgcolor=paleta['fundo'],
    font=dict(color=paleta['azul_escuro']),
    title_font=dict(size=18, color=paleta['azul_escuro']),
    width=1000,
    height=500,
    xaxis=dict(
        tickformat='%Y',
        dtick="M12",
        showgrid=True,
        gridcolor=paleta['cinza_claro'],
        linecolor=paleta['cinza_texto'],
        tickfont=dict(color=paleta['azul_escuro'])
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor=paleta['cinza_claro'],
        linecolor=paleta['cinza_texto'],
        tickfont=dict(color=paleta['azul_escuro'])
    )
)

fig1_p2.show()

# 3) Existem perfis de pacientes que apresentam maior demora para a confecção da FAV?

In [11]:
fig = px.scatter(
    df_perfil,
    x='idade',
    y='tempo_entre_fav_e_primeira_dialise',
    color='genero',
    labels={
        'idade': 'Idade',
        'tempo_entre_fav_e_primeira_dialise': 'Tempo de espera (dias)',
        'genero': 'Gênero'
    },
    title='Perfil dos Pacientes vs Tempo de Espera para Confecção da FAV',
    color_discrete_map={
        'Feminino': '#0077B6',
        'Masculino': '#90E0EF'
    }
)

# Layout com paleta médica
fig.update_layout(
    plot_bgcolor=paleta['fundo'],
    paper_bgcolor=paleta['fundo'],
    font=dict(color=paleta['azul_escuro']),
    title_font=dict(size=18, color=paleta['azul_escuro']),
    xaxis=dict(
        title='Idade',
        gridcolor=paleta['cinza_claro'],
        linecolor=paleta['cinza_texto']
    ),
    yaxis=dict(
        title='Tempo de espera (dias)',
        gridcolor=paleta['cinza_claro'],
        linecolor=paleta['cinza_texto']
    )
)

fig.show()

# 4) Existe variação no tempo de espera para a confecção da FAV entre diferentes unidades hospitalares de Porto Alegre?

In [24]:
df_espera['hospital'] = df_espera['hospital'].astype(str)

top_25_maiores = df_espera.sort_values(by='tempo_entre_fav_e_primeira_dialise', ascending=False).head(25)

top_25_menores = df_espera.sort_values(by='tempo_entre_fav_e_primeira_dialise', ascending=True).head(25)

df_extremos = pd.concat([top_25_maiores, top_25_menores]).reset_index(drop=True)
df_extremos[['hospital', 'idade', 'genero', 'tempo_entre_fav_e_primeira_dialise', 'data_primeira_dialise', 'data_atendimento_fav']]
df_extremos.sample(5)

,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav,genero,idade,hospital,tempo_entre_fav_e_primeira_dialise,tempo_anos
25,2016-10-31,cronico,2017-02-01,0,2016-11-01,M,36,2266474.0,1,0.002738
44,2021-05-31,cronico,2021-08-01,0,2021-06-01,M,50,2253046.0,1,0.002738
28,2018-01-31,cronico,2018-04-01,0,2018-02-01,F,53,2252295.0,1,0.002738
43,2021-07-31,cronico,2021-10-01,0,2021-08-01,M,56,2707918.0,1,0.002738
2,2014-01-31,cronico,2024-02-01,0,2024-08-01,F,37,2262509.0,3835,10.499658


In [ ]:
df_extremos['grupo'] = ['Maior Tempo'] * 25 + ['Menor Tempo'] * 25

# Criar uma coluna identificadora para o paciente (indexado)
df_extremos['id_paciente'] = df_extremos.index.astype(str)

# Construir matriz pivô: linhas = hospital, colunas = id_paciente, valores = tempo de espera
heatmap_data = df_extremos.pivot(
    index='hospital',
    columns='id_paciente',
    values='tempo_entre_fav_e_primeira_dialise'
)

# Plotar o heatmap
fig1_p4 = px.imshow(
    heatmap_data,
    color_continuous_scale='Blues',
    labels=dict(color="Tempo de espera (dias)", x="Paciente", y="Hospital"),
    aspect='auto',
    title='Heatmap dos 25 Maiores e 25 Menores Tempos de Espera para FAV'
)

# Layout com paleta
fig1_p4.update_layout(
    plot_bgcolor=paleta['fundo'],
    paper_bgcolor=paleta['fundo'],
    font=dict(color=paleta['azul_escuro']),
    title_font=dict(size=18, color=paleta['azul_escuro']),
    xaxis=dict(side='bottom')
)

fig1_p4.show()

# DASH Dashboard

In [74]:
# === App Dash ===
app = Dash(__name__)

app.layout = html.Div(children=[
    html.H1(children='Dashboard - Tempo de Espera na FAV'),

    html.Div(children='''
        Análise dos tempos de espera entre a primeira diálise e o atendimento na FAV.
    '''),

    dcc.Graph(
        id='bar-plot',
        figure=fig1_p1
    ),
    dcc.Graph(
        id='histogram',
        figure=fig2_p1
    ),

    dcc.Graph(
    id='line-plot',
    figure=fig1_p2
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)